# Divisão dos dados para validação de modelos

A validação de um modelo envolve avaliar a qualidade das suas previsões em dados não utilizados durante o treinamento.

Classicamente no aprendizado estatístico, esse “conjunto de previsão”, também chamado de “conjunto de validação” ou “conjunto de teste” de acordo com o contexto, é amostrado **aleatoriamente** de todos os dados disponíveis. Isso é chamado de divisão aleatória dos dados (em inglês, *random split*)

Veja, por exemplo, as muitas opções disponíveis no [scikit learn](https://scikit-learn.org/stable/modules/classes.html#module-sklearn.model_selection), que pode amostrar dados de forma completamente aleatória, ou levar em consideração a estratificação dos valores alvo (garantindo a mesma distribuição de classes nos conjuntos de treinamento e teste), bem como permitir o uso de mais de uma divisão dos dados (validação cruzada k-fold).

As métricas obtidas a partir de tais divisões aleatórias podem ser preditivas para o desempenho de um modelo em tarefas como reconhecimento de dígitos, por exemplo, para as quais não esperamos uma significativa mudança nos dados futuros (*data drift*), ou seja, casos em que é razoável supor que dados futuros serão semelhantes aos dados disponíveis durante o treinamento.

Este, no entanto, normalmente não é o caso para conjuntos de dados em quiminformática, especialmente quando, por exemplo, estamos tentando identificar novos resultados para um alvo de interesse. Conjuntos de teste aleatórios tendem a ser “mais fáceis” de prever do que dados futuros reais, o que significa que quaisquer métricas de qualidade calculadas não refletirão o verdadeiro desempenho do modelo. Abordagens alternativas são necessárias para explicar uma possível mudança no espaço químico dos novos compostos.

Vários métodos alternativos de divisão dos dados foram desenvolvidos. O [Deepchem](https://github.com/deepchem/deepchem/tree/master), um pacote em Python com ferramentas de aprendizado de máquina molecular, implementa alguns dos mais utilizados.

Para usar o DeepChem localmente em um Jupyter Notebook, siga as instruções em: https://deepchem.readthedocs.io/en/latest/get_started/installation.html#jupyter-notebook. Em seguida, instale o Jupyter usando:

`pip install notebook`

In [1]:
import deepchem as dc

dc.__version__

Skipped loading some Pytorch utilities, missing a dependency. No module named 'torch'


This module requires PyTorch to be installed.


No normalization for AvgIpc. Feature removed!
Skipped loading some PyTorch models, missing a dependency. No module named 'torch'
No module named 'torch'
Skipped loading modules with pytorch-geometric dependency, missing a dependency. No module named 'torch'
Skipped loading modules with pytorch-lightning dependency, missing a dependency. No module named 'torch'
Skipped loading some Jax models, missing a dependency. No module named 'jax'


'2.7.2.dev'

Vamos usar como exemplo o dataset BBBP do MoleculeNet, que apresenta como valores alvo a permeabilidade na barreira hematoencefálica (0 para não permeável, 1 para permeável).

In [2]:
# importar datasets
import pandas as pd 

# SMILES e valores alvo
url = 'https://raw.githubusercontent.com/rflameiro/Python_e_Quiminformatica/main/datasets/BBBP_curated.csv'
df_smi = pd.read_csv(url, sep=";", index_col=False)
# Fingerprints e valores alvo
url = 'https://raw.githubusercontent.com/rflameiro/Python_e_Quiminformatica/main/datasets/BBBP_morganFP_1024_radius3.csv'
df_fp = pd.read_csv(url, sep=";", index_col=False)

In [3]:
df_smi.columns

Index(['std_smiles', 'p_np'], dtype='object')

In [4]:
# Criamos a coluna "ids" contendo os SMILES para usar no DeepChem
df_fp["ids"] = df_smi["std_smiles"]

In [5]:
df_fp.head()

,morgan_bit_0,morgan_bit_1,morgan_bit_2,morgan_bit_3,morgan_bit_4,morgan_bit_5,morgan_bit_6,morgan_bit_7,morgan_bit_8,morgan_bit_9,...,morgan_bit_1016,morgan_bit_1017,morgan_bit_1018,morgan_bit_1019,morgan_bit_1020,morgan_bit_1021,morgan_bit_1022,morgan_bit_1023,target,ids
0,0,1,0,0,0,1,0,0,0,0,...,0,0,0,1,0,0,0,0,1,Cc1onc(-c2ccccc2Cl)c1C(=O)NC1C(=O)N2C1SC(C)(C)...
1,0,1,1,0,0,1,0,0,0,0,...,0,1,0,1,0,0,0,0,1,CCN1CCN(C(=O)NC(C(=O)NC2C(=O)N3C(C(=O)O)=C(CSc...
2,0,0,0,0,1,0,0,0,0,0,...,0,0,0,1,0,0,0,0,1,CN(C)C1C(=O)C(C(=O)NCN2CCCC2)C(=O)[C@@]2(O)C(=...
3,0,0,0,0,0,1,0,0,0,0,...,0,0,0,1,0,1,0,0,1,Cc1nccn1CC1CCc2c(C1=O)c1ccccc1n2C
4,0,0,0,0,0,1,0,1,0,0,...,0,1,0,1,0,0,0,0,1,COc1ccc([C@@H]2Sc3ccccc3N(CCN(C)C)C(=O)C2OC(C)...


In [6]:
print(df_smi.shape)
print(df_fp.shape)

(1934, 2)
(1934, 1026)


In [7]:
# Criar objeto NumpyDataset do Deepchem com os SMILES na coluna "ids"
X_cols = df_fp.columns.to_list()[:-2]
dataset = dc.data.NumpyDataset.from_dataframe(df_fp, 
                                              X=X_cols, 
                                              y="target", 
                                              ids="ids")

# Divisão aleatória (*Random split*)

Vamos criar uma divisão treinamento/teste aleatória para comparar com os outros métodos.

In [8]:
import deepchem as dc

splitter = dc.splits.RandomSplitter()
train_random, test_random = splitter.train_test_split(dataset)

In [9]:
train_random.get_shape()

((1547, 1024), (1547, 1), (1547, 0), (1547,))

In [10]:
test_random.get_shape()

((387, 1024), (387, 1), (387, 0), (387,))

In [11]:
# É possível converter de volta a pandas DataFrames
# Note que os rótulos das colunas X são substituídos por X1, X2...
pandas_random = train_random.to_dataframe()
pandas_random.shape

(1547, 1026)

In [12]:
pandas_random.head()

,X1,X2,X3,X4,X5,X6,X7,X8,X9,X10,...,X1017,X1018,X1019,X1020,X1021,X1022,X1023,X1024,y,ids
0,0,0,0,0,0,1,0,0,0,0,...,0,1,0,1,0,0,0,0,0,CC(=O)OCC1=C(C(=O)O)N2C(=O)C(NC(=O)CC#N)C2SC1
1,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,O=C(Nc1ccccc1)OCC1(COC(=O)Nc2ccccc2)CCCC1
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,1,0,0,1,CN1CCC(=C2c3c(cccc3)Sc3c2cccc3)CC1
3,0,0,0,0,0,0,0,0,0,0,...,0,1,0,1,0,0,0,0,1,O=[N+]([O-])C1=CC=NC1NCCSCc1ncccc1Br
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,CN(C)c1nc(O)c(-c2ccccc2)o1


In [13]:
# Alternativamente, exporte como .csv
# train_random.to_csv('out.csv')

# Divisão por agrupamentos (*Cluster split*)

Existem duas versões desta divisão:

A primeira requer o uso de algoritmos de agrupamento (*clustering*) que tomam como entrada um número predefinido de *clusters* finais. Se você quiser, por exemplo, criar uma divisão 80:20 dos dados, poderá usar o algoritmo *K-means* com o número de *clusters* = 5. Um exemplo da literatura pode ser encontrado [neste artigo](https://pubs.rsc. org/en/content/articlelanding/2019/sc/c8sc04175j), que usa "agrupamento K-means com K = 5 em MACCS fingerprints". Para mais informações sobre métodos de agrupamento, veja o [Notebook Clustering (Agrupamento)](https://github.com/rflameiro/Python_e_Quiminformatica/blob/main/Quiminformatica/Clustering%20(Agrupamento).ipynb)

A segunda versão requer algoritmos de agrupamento que usam um valor de corte predefinido para criar os agrupamentos. Neste caso, não podemos saber de antemão o número final de *clusters*, apenas controlar se haverá mais ou menos deles, definindo o valor de corte. Para mais detalhes, veja o [Notebook Clustering (Agrupamento) - Butina](https://github.com/rflameiro/Python_e_Quiminformatica/blob/main/Quiminformatica/Clustering%20(Agrupamento)%20-%20Butina.ipynb).

No Deepchem, o método disponível é do segundo tipo, e também emprega o algoritmo de agrupamento Butina do RDKit, que é otimizado para agrupamento de *fingerprints* moleculares. O método requer a instalação do RDKit, e usa os SMILES como entrada.

In [14]:
import deepchem as dc

butinasplitter = dc.splits.ButinaSplitter()
train_butina, test_butina = butinasplitter.train_test_split(dataset)

In [15]:
train_butina.get_shape()

((1547, 1024), (1547, 1), (1547, 0), (1547,))

In [16]:
test_butina.get_shape()

((387, 1024), (387, 1), (387, 0), (387,))

In [17]:
train_butina.to_dataframe().head()

,X1,X2,X3,X4,X5,X6,X7,X8,X9,X10,...,X1017,X1018,X1019,X1020,X1021,X1022,X1023,X1024,y,ids
0,0,0,0,0,0,0,0,0,0,0,...,0,1,0,1,0,0,0,0,1,CC(=O)OCC(=O)[C@@]1(O)CC[C@H]2[C@@H]3CC=C4CC(=...
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,1,0,0,0,0,C[C@H]1C[C@H]2[C@@H]3CC=C4CC(=O)C=C[C@]4(C)[C@...
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,C[C@]12C[C@H](O)[C@H]3[C@@H](CC=C4CC(=O)C=C[C@...
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,C[C@]12CC(=O)C3[C@@H](CC=C4CC(=O)C=C[C@@]43C)[...
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,1,0,0,0,0,C[C@]12C[C@H](O)[C@@]3(F)[C@@H](CC=C4CC(=O)C=C...


# Divisão por *scaffolds* (*Scaffold split*)

Consiste em dividir os dados de acordo com os *scaffolds* Bemis-Murcko de forma que moléculas com o mesmo *scaffold* estejam todas ou no conjunto de treinamento ou no conjunto de teste, mas não em ambos. Normalmente, compostos pertencentes aos *scaffolds* mais raros são colocados no conjunto de teste. Este método pode não ser capaz de criar divisões de dados em proporções exatas porque depende da distribuição de estruturas no conjunto de dados.

Você pode encontrar mais informações, em inglês, em: 

https://www.blopig.com/blog/2021/06/out-of-distribution-generalisation-and-scaffold-splitting-in-molecular-property-prediction/

https://practicalcheminformatics.blogspot.com/2023/06/getting-real-with-molecular-property.html

A implementação do Deepchem requer a instalação do RDKit e usa SMILES como entrada.

In [18]:
import deepchem as dc

scaffoldsplitter = dc.splits.ScaffoldSplitter()
train_scaffold, test_scaffold, = scaffoldsplitter.train_test_split(dataset)

In [19]:
train_scaffold.get_shape()

((1547, 1024), (1547, 1), (1547, 0), (1547,))

In [20]:
test_scaffold.get_shape()

((387, 1024), (387, 1), (387, 0), (387,))

In [21]:
train_scaffold.to_dataframe().head()

,X1,X2,X3,X4,X5,X6,X7,X8,X9,X10,...,X1017,X1018,X1019,X1020,X1021,X1022,X1023,X1024,y,ids
0,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,C[C@H](N)Cc1ccccc1
1,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,CS(=O)(=O)c1ccc([C@@H](O)[C@@H](CO)NC(=O)C(Cl)...
2,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,Cc1c(OCC(C)N)c(C)ccc1
3,0,1,0,0,1,0,0,0,0,0,...,0,1,0,0,0,0,0,0,1,CCCC(=O)Nc1cc(C(C)=O)c(OCC(O)CNC(C)C)cc1
4,1,1,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,1,CCN(CC)C(=O)Nc1ccc(OCC(O)CNC(C)(C)C)c(C(C)=O)c1


# Divisão por *fingerprints* (*Fingerprint split*)

A divisão é baseada na semelhança (de acordo com o índice de Tanimoto, por exemplo) entre *fingerprints* moleculares, como ECFP4. O algoritmo procura dividir os dados de forma que as moléculas em um conjunto sejam tão diferentes quanto possível daquelas nos outros conjuntos.

Isto é semelhante à abordagem "Divisão de vizinhos" (*Neighbor split*), na qual o número de vizinhos de cada composto é calculado e, em seguida, compostos com menos vizinhos são colocados no conjunto de teste. Um vizinho pode ser definido como um composto com similaridade maior que um limite predefinido (0,5-0,7) de acordo com alguma métrica de similaridade (Tanimoto/Cosine/Dice) nos *fingerprints* moleculares.

Esteja ciente de que esses métodos provavelmente criarão conjuntos de testes muito difíceis e, consequentemente, as métricas calculadas poderão subestimar o verdadeiro desempenho do modelo.

In [22]:
import deepchem as dc

fingerprintsplitter = dc.splits.FingerprintSplitter()
train_fp, test_fp = fingerprintsplitter.train_test_split(dataset)

In [23]:
train_fp.get_shape()

((1547, 1024), (1547, 1), (1547, 0), (1547,))

In [24]:
test_fp.get_shape()

((387, 1024), (387, 1), (387, 0), (387,))

In [25]:
train_fp.to_dataframe().head()

,X1,X2,X3,X4,X5,X6,X7,X8,X9,X10,...,X1017,X1018,X1019,X1020,X1021,X1022,X1023,X1024,y,ids
0,0,1,0,0,0,1,0,0,0,0,...,0,0,0,1,0,0,0,0,1,Cc1onc(-c2ccccc2Cl)c1C(=O)NC1C(=O)N2C1SC(C)(C)...
1,0,0,0,0,0,1,0,1,0,0,...,0,1,0,1,0,0,0,0,1,COc1ccc([C@@H]2Sc3ccccc3N(CCN(C)C)C(=O)C2OC(C)...
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,N=C(N)NC(=O)c1nc(Cl)c(N)nc1N
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,Nc1nnc(-c2cccc(Cl)c2Cl)c(N)n1
4,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,CCCC(C)C


# Divisão temporal (*Temporal split*)

Este método consiste em realizar uma divisão treinamento/teste alocando instâncias mais recentes no conjunto de teste. Essa abordagem é diferente do [Time Series Split](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.TimeSeriesSplit.html) do scikit-learn, que pode gerar K-splits para dados de séries temporais.

Em [Time-Split Cross-Validation as a Method for Estimating the Goodness of Prospective Prediction (2013)](https://pubs.acs.org/doi/10.1021/ci400084k), Sheridan mostra que usar uma única divisão para alocar 10, 25 ou 50% dos compostos "mais novos" no conjunto de teste prevê corretamente o valor de R² para dados futuros, enquanto uma divisão aleatória tende a ser muito otimista em relação ao desempenho do modelo.

Esta implementação do Deepchem pressupõe que seu conjunto de dados esteja ordenado corretamente, com compostos mais recentes na parte inferior do conjunto de dados. Portanto, corresponde a uma simples divisão de acordo com o índice.

In [26]:
import deepchem as dc

indexsplitter = dc.splits.IndexSplitter()
train_dataset, test_dataset = indexsplitter.train_test_split(dataset, frac_train=0.8)

In [27]:
# É o mesmo que usar:
# X_train, X_test, y_train, y_test = train_test_split(
#     X, y, test_size=0.2, shuffle=False)

# Mais opções no DeepChem

Além do `train_test_split`, alguns dos outros métodos disponíveis no DeepChem incluem `train_valid_test_split`, que cria um conjunto de validação adicional (treinamento/validação/teste), e `k_fold_split`, que cria várias divisões para uso em validação cruzada (observe que isso nem sempre fará fazer sentido, como para dados de séries temporais).

# SIMPD - Simulated Time Split

A divisão temporal tem se mostrado uma boa escolha para prever o desempenho futuro de modelos. No entanto, as informações sobre as datas de síntese/teste raramente estão disponíveis, especialmente para dados públicos. Em [SIMPD: an Algorithm for Generating Simulated Time Splits for Validating Machine Learning Approaches (2023)](https://chemrxiv.org/engage/chemrxiv/article-details/6406049e6642bf8c8f10e189), Landrum et al. descrevem o algoritmo SIMPD, que usa uma abordagem de algoritmo genético para criar uma "divisão temporal aproximada" para qualquer conjunto de dados.

Os autores identificaram que essas divisões temporais simuladas refletem as propriedades esperadas para divisões temporais verdadeiras: descritores que tendem a aumentar ao longo de um projeto de química medicinal, como a “acessibilidade sintética” (os compostos ficam mais complexos), contagem de átomos pesados e área superficial polar topológica (TPSA) também aumentam nos conjuntos de teste selecionados em relação aos conjuntos de treinamento.

O algoritmo foi disponibilizado no [GitHub](github.com/rinikerlab/molecular_time_series), mas não achei muito fácil de implementar. Parece que o SIMPD poderá ser adicionado ao [RDKit - Prefer](https://github.com/rdkit/PREFER) em breve, então vou guardá-lo para um futuro Notebook.

# MUV

Em [Maximum Unbiased Validation (MUV) Data Sets for Virtual Screening Based on PubChem Bioactivity Data (2009)](https://pubs.acs.org/doi/10.1021/ci8002649), Rohren e Baumann apresentam o *workflow* MUV para criar conjuntos de dados não-enviesados para o treinamento de modelos para triagem virtual.

Não consegui encontrar o *workflow* MUV, então vamos verificar a próxima abordagem, que é baseada nele.

# AVE

Em [Most Ligand-Based Classification Benchmarks Reward Memorization Rather than Generalization (2018)](https://pubs.acs.org/doi/abs/10.1021/acs.jcim.7b00403), Wallach e Heifets investigam o fenômeno do "sobreajuste não detectado" (*undetected overfitting*), que é consequência da existência de informações redundantes entre os conjuntos de treinamento e de teste, ou seja, semelhanças entre compostos ativo-ativo e inativo-inativo nos dois conjuntos. Inspirada na abordagem MUV, a medida de redundância AVE, ou **viés AVE** (*AVE bias*), foi proposta para medir o viés do conjunto de validação. Os autores afirmam que valores maiores de *AVE bias* correlacionam com melhores métricas de desempenho dos modelos, mas sem que essas métricas melhores reflitam de fato modelos mais preditivos, pois conjuntos de treinamento enviesados são mais fáceis de prever.

Os autores demonstram que mesmo dados temporais podem apresentar um viés significativo, pois há um viés intrínseco nos compostos que são selecionados para síntese.

Em resumo, o *AVE bias* descreve a capacidade de um modelo 1-NN (*1-nearest neighbor*) prever um conjunto de validação. 

Além da medida de viés, os autores também propõem o uso de um algoritmo  genético para separação de dados que minimiza o viés, que chamaremos de *AVE split*.

Tenha em mente que esse *split* também tém suas limitações, pois não fornecerá necessariamente um *split* ótimo, e pode introduzir vieses não esperados. De qualquer forma, os autores sugerem medir o viés do seu conjunto de teste e sempre comparar a performance do seu modelo com o 1-NN.

Dois scripts em Python foram disponibilizados pelos autores:
`analyze_AVE_bias.py` mede o viés de um dataset que já foi separado em treinamento e teste (os inputs devem estar no formato .smi, separado por tab ou espaço). 
`remove_AVE_bias.py` pega um conjunto de dados inteiro e cria uma divisão ideal usando o algoritmo proposto.

Atualizei os scripts para funcionar com Python 3 e corrigi alguns erros de indentação. Os arquivos atualizados podem ser encontrados [nesta pasta](https://github.com/rflameiro/Python_e_Quiminformatica/tree/main/modules). Usarei as opções padrão dos scripts neste exemplo, mas dê uma olhada em cada um caso tenha interesse em ver todas as opções disponíveis.

## analyze_AVE_bias

Vamos comparar a divisão aleatória (*random*) com a por *scaffolds*. O viés AVE é o output final do script, mostrado como: (AA-AI)+(II-IA)

In [28]:
# Converter a pandas DataFrame
train_random_df = train_random.to_dataframe()
test_random_df = test_random.to_dataframe()

train_scaffold_df = train_scaffold.to_dataframe()
test_scaffold_df = test_scaffold.to_dataframe()

In [31]:
def dataframes_to_smi(df_train, df_test, label=""):
    """
    Função para escrever arquivos .smi como inputs para o script analyze_AVE_bias.py
    df_train, df_test: Pandas DataFrame com uma coluna SMILES rotulada como "ids" e a coluna de destino binária "y"
    label: será usado para nomear os arquivos .smi gerados. Use-o para comparar divisões,
        por exemplo, "aleatório", "scaffold", "temporal"

    Modifiquei a função para mostrar os resultados dentro do notebook, usando print().
    Caso prefira criar um .txt com os resultados, você pode modificar o script (des)comentando onde indicado
    """
    active_training = df_train[df_train["y"] == 1]["ids"].to_list()
    inactive_training = df_train[df_train["y"] == 0]["ids"].to_list()
    active_test = df_test[df_test["y"] == 1]["ids"].to_list()
    inactive_test = df_test[df_test["y"] == 0]["ids"].to_list()

    active_training_path = "active_training_" + label + ".smi"   
    with open(active_training_path,'w+') as file:
        file.write(' \n'.join(active_training))
    print(active_training_path)

    inactive_training_path = "inactive_training_" + label + ".smi"   
    with open(inactive_training_path,'w+') as file:
        file.write(' \n'.join(inactive_training))
    print(inactive_training_path)
    
    active_test_path = "active_test_" + label + ".smi"   
    with open(active_test_path,'w+') as file:
        file.write(' \n'.join(active_test))
    print(active_test_path)

    inactive_test_path = "inactive_test_" + label + ".smi"   
    with open(inactive_test_path,'w+') as file:
        file.write(' \n'.join(inactive_test))
    print(inactive_test_path)


In [32]:
# Criar .smi files - random split
dataframes_to_smi(train_random_df, test_random_df, label="random")

active_training_random.smi
inactive_training_random.smi
active_test_random.smi
inactive_test_random.smi


Note: I modified the script to print the results on the Notebook, so the `-outFile` argument will not be used for anything, but it is still required to run the script.

In [58]:
# run script
! python analyze_AVE_bias.py -activeMolsTraining active_training_random.smi -inactiveMolsTraining inactive_training_random.smi -activeMolsTesting active_test_random.smi -inactiveMolsTesting inactive_test_random.smi -outFile ave_results_random.txt

#ActTrain = 1177 
#InactTrain = 370 
#ActTest = 299 
#InactTest = 88 
knn1 = 0.834 
lr = 0.909 
rf = 0.942 
svm = 0.921 
AA-AI = 0.221 
II-IA = 0.169 
(AA-AI)+(II-IA) = 0.390


[15:38:20] WARNING: no name column found on line 1176
[15:38:20] WARNING: no name column found on line 369
[15:38:20] WARNING: no name column found on line 298
[15:38:20] WARNING: no name column found on line 87
C:\Users\rafae\miniconda3\envs\deepchem-test\lib\site-packages\sklearn\metrics\pairwise.py:2182: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)


In [34]:
# Criar .smi files - scaffold split
dataframes_to_smi(train_scaffold_df, test_scaffold_df, label="scaffold")

active_training_scaffold.smi
inactive_training_scaffold.smi
active_test_scaffold.smi
inactive_test_scaffold.smi


In [59]:
# run script
! python analyze_AVE_bias.py -activeMolsTraining active_training_scaffold.smi -inactiveMolsTraining inactive_training_scaffold.smi -activeMolsTesting active_test_scaffold.smi -inactiveMolsTesting inactive_test_scaffold.smi -outFile ave_results_scaffold.txt

#ActTrain = 1274 
#InactTrain = 273 
#ActTest = 202 
#InactTest = 185 
knn1 = 0.712 
lr = 0.818 
rf = 0.839 
svm = 0.837 
AA-AI = 0.158 
II-IA = 0.048 
(AA-AI)+(II-IA) = 0.205


[15:40:28] WARNING: no name column found on line 1273
[15:40:28] WARNING: no name column found on line 272
[15:40:28] WARNING: no name column found on line 201
[15:40:29] WARNING: no name column found on line 184
C:\Users\rafae\miniconda3\envs\deepchem-test\lib\site-packages\sklearn\metrics\pairwise.py:2182: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)


Como podemos ver, o viés da divisão por *scaffolds* (0,205) é menor que o da divisão aleatória (0,390). Observe como a divisão por *scaffolds* diminui tanto os valores de AA-AI como de II-IA.

AA-AI é uma medida de quão aglomerados estão os ativos dos conjuntos de teste e os ativos do conjunto de treinamento, e II-IA, da mesma forma, mede a aglomeração entre os inativos dos dois conjuntos. Isso significa que, embora a divisão por *scaffolds* seja capaz de tornar os ativos do conjunto de teste menos semelhantes aos ativos do conjunto de treinamento, seu principal ponto forte neste exemplo foi diminuir significativamente as semelhanças inativo-inativo entre os conjuntos de teste de treinamento.

## remove_AVE_bias

Vamos ver se conseguimos uma melhoria em relação ao *scaffold split*.

In [36]:
def dataframe_to_smi(df, label=""):
    """
    Function to write .smi inputs to remove_AVE_bias.py
    df: Pandas DataFrame with a SMILES column labeled as "ids", and the binary target column "y"
    label: will be used to name the generated .smi files.
    """
    active_mols = df[df["y"] == 1]["ids"].to_list()
    inactive_mols = df[df["y"] == 0]["ids"].to_list()

    active_mols_path = "active_mols_" + label + ".smi"   
    with open(active_mols_path,'w+') as file:
        file.write(' \n'.join(active_mols))
    print(active_mols_path)

    inactive_mols_path = "inactive_mols_" + label + ".smi"   
    with open(inactive_mols_path,'w+') as file:
        file.write(' \n'.join(inactive_mols))
    print(inactive_mols_path)

In [38]:
df = df_smi.copy()
df.columns = ["ids", "y"]
df.head()

,ids,y
0,Cc1onc(-c2ccccc2Cl)c1C(=O)NC1C(=O)N2C1SC(C)(C)...,1
1,CCN1CCN(C(=O)NC(C(=O)NC2C(=O)N3C(C(=O)O)=C(CSc...,1
2,CN(C)C1C(=O)C(C(=O)NCN2CCCC2)C(=O)[C@@]2(O)C(=...,1
3,Cc1nccn1CC1CCc2c(C1=O)c1ccccc1n2C,1
4,COc1ccc([C@@H]2Sc3ccccc3N(CCN(C)C)C(=O)C2OC(C)...,1


In [39]:
dataframe_to_smi(df, label="remove_bias")

active_mols_remove_bias.smi
inactive_mols_remove_bias.smi


In [40]:
# remove_AVE_bias.py
! python remove_AVE_bias.py -activeMols active_mols_remove_bias.smi -inactiveMols inactive_mols_remove_bias.smi

read 1476 actives and 458 inactives
calc aa_D_ref
calc ii_D_ref
calc ai_D_ref
done
calculate objectives for the population
remove similar sets
removing 0 similar sets
population size after similarity filter:  100
select the next generation
iter= 1 fullPopObj= 0.367 topPopObj= 0.337 finalPopObj= 0.311 minObj= 99999
breed
calculate objectives for the population
remove similar sets
removing 0 similar sets
population size after similarity filter:  100
select the next generation
iter= 2 fullPopObj= 0.341 topPopObj= 0.315 finalPopObj= 0.297 minObj= 0.311
breed
calculate objectives for the population
remove similar sets
removing 0 similar sets
population size after similarity filter:  100
select the next generation
iter= 3 fullPopObj= 0.318 topPopObj= 0.291 finalPopObj= 0.262 minObj= 0.297
breed
calculate objectives for the population
remove similar sets
removing 0 similar sets
population size after similarity filter:  100
select the next generation
iter= 4 fullPopObj= 0.298 topPopObj= 0.271 

[12:51:24] WARNING: no name column found on line 1475
[12:51:24] WARNING: no name column found on line 457


Now let's see how the bias of this AVE split compares to that of the other splits:

In [60]:
! python analyze_AVE_bias.py -activeMolsTraining actives.T.smi -inactiveMolsTraining inactives.T.smi -activeMolsTesting actives.V.smi -inactiveMolsTesting inactives.V.smi -outFile ave_results_AVE_split.txt

#ActTrain = 1174 
#InactTrain = 360 
#ActTest = 290 
#InactTest = 83 
knn1 = 0.523 
lr = 0.747 
rf = 0.738 
svm = 0.758 
AA-AI = 0.160 
II-IA = -0.158 
(AA-AI)+(II-IA) = 0.002


C:\Users\rafae\miniconda3\envs\deepchem-test\lib\site-packages\sklearn\metrics\pairwise.py:2182: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)


O viés geral é muito menor do que o do *scaffold split*. Porém, observe como o fator AA-AI é aproximadamente o mesmo, enquanto II-IA tornou-se negativo. Isso significa que os inativos do conjunto de teste são geralmente mais semelhantes aos ativos de treinamento do que aos inativos de treinamento, e que a classificação dos inativos será mais desafiadora. Isso pode ser um problema ou não, dependendo do seu uso do modelo.

## Resultados

| Método de divisão | Viés | n° ativos treinamento | n° inativos treinamento | n° ativos teste | n° inativos teste |
|---|---|---|---|---|---|
| Divisão aleatória (Random split) | 0.390 | 1177 | 370 | 299  | 88 |
| Divisão por scaffolds (Scaffold split) | 0.205 | 1274 | 273 | 202 | 185 |
| Divisão pelo método AVE (AVE split) | 0.002 | 1174 | 360 | 290 | 83 |

Em resumo, a divisão por *scaffolds* é menos enviesada do que a divisão aleatória, conforme esperávamos, mas ainda tem um viés significativo quando comparada à divisão pelo método AVE. No artigo, valores de viés na faixa de 0,1-0,2 já indicam conjuntos de dados que podem ser facilmente "resolvidos" por todos os algoritmos testados (LR, SVM, RF, 1-NN), alcançando valores de ROC/AUC > 0,9.

Em nosso exemplo, o viés vem principalmente das semelhanças entre os ativos de treinamento/teste, e esse viés não foi removido pela divisão AVE. A similaridade entre os inativos, por outro lado, foi significativamente alterada pela divisão, que chegou a tornar mais difícil diferenciar os inativos de teste dos ativos de treinamento.

Como não foram perdidos muitos compostos no processo, pode ser uma boa ideia usar a divisão AVE neste caso. Espere, no entanto, uma queda no desempenho do seu modelo, uma vez que alguns inativos do conjunto de teste provavelmente serão classificados erroneamente como ativos. Você pode tentar explorar diferentes representações moleculares para superar esse problema.